In [2]:
#### first 2 are connected to google.colab ####
### 3. Install dependencies ###
# !apt update
# !apt install -y tesseract-ocr libtesseract-dev
!pip install pytesseract Pillow easyocr boto3


  Using cached pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
  Using cached easyocr-1.7.2-py3-none-any.whl.metadata (10 kB)
  Using cached opencv_python_headless-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached ninja-1.13.0-py3-none-win_amd64.whl.metadata (5.1 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
Using cached pytesseract-0.3.13-py3-none-any.whl (14 kB)
Using cached easyocr-1.7.2-py3-none-any.whl (2.9 MB)
Using cached ninja-1.13.0-py3-none-win_amd64.whl (309 kB)
Using cached opencv_python_headless-4.12.0.88-cp37-abi3-win_amd64.whl (38.9 MB)
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.9 MB 7.2 MB/s eta 0:00:02
   -------- ------------------------------- 2.6/12.9 MB 7.2 MB/s eta 0:00:02
   ------------- -------------------------- 4.5/12.9 MB 7.7 MB/s eta 0:00:02
   -------------------- ------------------- 6.6/12.9 MB 8.2 MB/s eta 0:00:01
   ----------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.2.6 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
label-studio 1.16.0 requires numpy<2.0.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
label-studio 1.16.0 requires wheel<=0.40.0,>=0.38.1, but you have wheel 0.45.1 which is incompatible.
label-studio-sdk 1.0.10 requires numpy<2.0.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
langchain 0.2.17 requires langchain-core<0.3.0,>=0.2.43, but you have langchain-core 0.3.60 which is incompatible.
langchain 0.2.17 requires numpy<2,>=1; python_version < "3.12", but you have numpy 2.2.6 which is incompatible.
matplotlib 3.8.2 requires numpy<2,>=1.21, but you have numpy 2.2.6 which is incompatible.
mediapipe 0.10.21 requi

In [3]:
import pytesseract
from PIL import Image
# 4. Text detection
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"


In [10]:
import os
import boto3
import pytesseract
from PIL import Image
from easyocr import Reader
from dotenv import load_dotenv

load_dotenv()
reader = Reader(['en'])
image_path = './data/exit.jpg'

aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")

def read_text_tesseract(image_path):

    text = pytesseract.image_to_string(Image.open(image_path), lang='eng')
    return text

def read_text_easyocr(image_path):

    text = ''
    result = reader.readtext(Image.open(image_path))
    for results in result:
        text = text+results[1] + ' '

    text = text[:-1]
    return text

def read_text_textract(image_path):
    textract_client = boto3.client('textract', aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name='us-east-1')

    with open(image_path, 'rb') as im:
        response = textract_client.detect_document_text(Document={'Bytes': im.read()})

    text = ''

    for item in response['Blocks']:
        if item['BlockType'] == 'LINE':
            text = text+item['Text']+ ' '

    text = text[:-1]
    return text


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [ ]:
# 5. Compare performances

def jaccard_similarity(sentence1, sentence2):

    text1 = set(sentence1.lower().split())
    text2 = set(sentence2.lower().split())

    intersection_size = len(text1.intersection(text2))
    union_size = len(text1.union(text2))

    similarity = intersection_size / union_size if union_size != 0 else 0.0

    return similarity

score_tesseract = 0
score_easyocr = 0
score_textract = 0

for image_path in os.listdir('./data'):
    image_path = os.path.join('./data', image_path)

    gt = image_path[:-4].replace('_', ' ').lower()
    score_tesseract+=jaccard_similarity(gt, read_text_tesseract(image_path).lower().replace('\n', '').replace('!', '').replace('?','').replace('.',''))
    score_easyocr+=jaccard_similarity(gt, read_text_easyocr(image_path).lower().replace('\n', '').replace('!', '').replace('?','').replace('.',''))
    score_textract+=jaccard_similarity(gt, read_text_textract(image_path).lower().replace('\n', '').replace('!', '').replace('?','').replace('.',''))


print('score tesseract',score_tesseract/139)
print('score easocr',score_easyocr/139)
print('score textract',score_textract/139)
